In [25]:
import pandas as pd
from glob import glob
import config
import utils
from importlib import reload
from os.path import join as j

reload(config)
reload(utils)

<module 'utils' from '/home/dauin_user/gattanasio/CTS/CTS/utils.py'>

In [21]:
CLASSIFIERS = {str(c) for c in config.HPARAMS.keys()}
print(CLASSIFIERS)

DETERMINISTIC = {str(c) for c in config.DETERMINISTIC}
print(DETERMINISTIC)

NUM_CLASSES = {2, 3}

CRYPTO = {str(c) for c in config.Cryptocurrency}
print(CRYPTO)

CLASS_RESULTS = j("results", "classification")
TRADING_RESULTS = j("results", "trading")

{'KNN', 'LG', 'RFC', 'GNB', 'MLP', 'SVC', 'MNB'}
{'MNB', 'GNB', 'KNN'}
{'LTC', 'BTC', 'ETH'}


In [30]:
def process_classification_results(data, num_classes):
    d = {
        "F1_macro": data.loc["f1-score", "macro avg"],
        "F1_weighted": data.loc["f1-score", "weighted avg"],
        "Accuracy": data.loc["precision", "accuracy"],
    }
    
    if num_classes == 2:
        d["F1_O"] = data.loc["f1-score", "1"]
        d["Precision_O"] = data.loc["precision", "1"]
        d["Recall_O"] = data.loc["recall", "1"]
    else:
        d["F1_O"] = data.loc["f1-score", ["-1", "1"]].mean()
        d["Precision_O"] = data.loc["precision", ["-1", "1"]].mean()
        d["Recall_O"] = data.loc["recall", ["-1", "1"]].mean()
    
    return d

In [99]:
results = list()

# read and process deterministic models
for model in DETERMINISTIC:
    for n_class in NUM_CLASSES:
        for crypto in CRYPTO:
            filename = utils.get_filename(crypto, model, n_class, None)
            res = pd.read_csv(j(CLASS_RESULTS, filename), index_col="metric")
            d = process_classification_results(res, n_class)
            d["classifier"] = model
            d["n_class"] = n_class
            d["crypto"] = crypto
            results.append(d)
            
deterministic = pd.DataFrame(results)

results = list()

# read and process non-deterministic models: average across seeds (random initializations)
for model in list(CLASSIFIERS - DETERMINISTIC) + ["LSTM"]:
    for n_class in NUM_CLASSES:
        for crypto in CRYPTO:
            for seed in range(10):
                filename = utils.get_filename(crypto, model, n_class, seed)
                
                if model == "LSTM":
                    res = pd.read_csv(j(CLASS_RESULTS, filename), index_col=0)
                    res.index.name = "metric"
                else:
                    res = pd.read_csv(j(CLASS_RESULTS, filename), index_col="metric")
                
                d = process_classification_results(res, n_class)
                d["classifier"] = model
                d["n_class"] = n_class
                d["crypto"] = crypto
                d["seed"] = seed
                results.append(d)
                
non_deterministic_mean = pd.DataFrame(results).groupby(["classifier", "n_class", "crypto"]).mean().drop("seed", axis=1).reset_index()
non_deterministic_std = pd.DataFrame(results).groupby(["classifier", "n_class", "crypto"]).std().drop("seed", axis=1).reset_index()

In [98]:
deterministic.pivot_table(
    index="classifier", columns=["n_class", "crypto"]
).drop(
    "Accuracy", axis=1
).to_csv(j("results", "classification", "comparison_det.csv"))

In [101]:
non_deterministic_mean.pivot_table(
    index="classifier", columns=["n_class", "crypto"]
).drop(
    "Accuracy", axis=1
).to_csv(j("results", "classification", "comparison_nondet_mean.csv"))

non_deterministic_std.pivot_table(
    index="classifier", columns=["n_class", "crypto"]
).drop(
    "Accuracy", axis=1
).to_csv(j("results", "classification", "comparison_nondet_std.csv"))